# Model 1

## Pakete und Daten laden

In einem ersten Schritt laden wir die nöten Pakete und laden die Daten.

In [ ]:
import pandas as pd
import stopwords
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import sklearn
import re
import spacy
import numpy as np
import random
import torch
from bertopic.vectorizers import ClassTfidfTransformer
import os
import openpyxl
import optuna
from sklearn.cluster import KMeans
from bertopic.representation import MaximalMarginalRelevance
from pathlib import Path

target = Path("C:/Users/mhu/Documents/github/topic_model_it")
os.chdir(target)

data_model_1b = pd.read_csv("data/informatikkurse.csv")
data_model_1b = data_model_1b[data_model_1b['model_1_b'] == 1].copy()
data_model_1b

c:\Users\mhu\miniconda3\envs\bertopic-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


   Unnamed: 0  veranstaltung_id  \
0           1           1717848   
1           2           1717914   
2           3           1717915   
3           4           1717989   
4           5           1717990   

                                 veranstaltung_titel kursbeschreibung  \
0    Übung zu: Didaktische Grundlagen für Informatik              NaN   
1  Seminar zu: Lehren und Lernen im Fach Informat...              NaN   
2                  Übung zu: Ethik in der Informatik              NaN   
3                  Übung zu: Ethik in der Informatik              NaN   
4                  infSP2F-01a: Softwareprojekt (2F)              NaN   

  kursformat_original sprache_original lernziele lernmethode literatur  \
0               Übung              NaN       NaN         NaN       NaN   
1             Seminar              NaN       NaN         NaN       NaN   
2               Übung              NaN       NaN         NaN       NaN   
3               Übung              NaN       NaN      

C:\Users\mhu\AppData\Local\Temp\ipykernel_12564\3578484799.py:25: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data_model_1b = pd.read_csv("data/informatikkurse.csv")


## Reproduzierbarkeit durch Seeds
Im folgenden fixieren wir die Zufallszahlen-Generatoren aller beteiligten Bibliotheken auf den Wert 11. Dies stellt sicher, dass die Experimente bei jedem Durchlauf identische Ergebnisse liefern.

In [2]:
seed = 11  # Initialisiert den Seed-Wert für reproduzierbare Ergebnisse
np.random.seed(seed)  # Setzt den Seed für NumPy-Zufallszahlengeneratoren
random.seed(seed)  # Setzt den Seed für den Python-eigenen Zufallszahlengenerator
torch.manual_seed(seed)  # Setzt den Seed für PyTorch-Zufallszahlen
if torch.cuda.is_available():  # Überprüft, ob CUDA (GPU-Unterstützung) verfügbar ist
    torch.cuda.manual_seed_all(seed)  # Setzt den Seed für alle CUDA-Zufallszahlen (für GPU-Berechnungen)

## Docs

Wir generieren als nächsten die Dokumente, die das Eingangsmaterial für das Topic-Model bilden.

In [3]:
# Anzeige der Spaltennamen von data_model_1b
print(data_model_1b.columns.tolist())

data_model_1b = data_model_1b[["veranstaltung_titel", "kursbeschreibung"]].copy()
data_model_1b.head()

data_model_1b = data_model_1b.apply(lambda x: x.fillna('') if x.dtype == 'O' else x)  # Ersetzt fehlende Werte durch leere Strings in Objektspalten (Strings) und belässt numerische Spalten unverändert
data_model_1b['titel_kursbesch'] = data_model_1b['veranstaltung_titel'] + ' ' + data_model_1b['kursbeschreibung']  # Kombiniert die Spalten "titel" und "kursbeschreibung" zu einer neuen Spalte "titel_kursbesch"
docs = data_model_1b['titel_kursbesch'].tolist()  # Konvertiert die Inhalte der Spalte "titel_kursbesch" in eine Liste von Strings

['Unnamed: 0', 'veranstaltung_id', 'veranstaltung_titel', 'kursbeschreibung', 'kursformat_original', 'sprache_original', 'lernziele', 'lernmethode', 'literatur', 'voraussetzungen', 'zusatzinformationen', 'anmerkungen', 'pruefung', 'dozierende', 'teilnehmerzahl', 'sws', 'ects', 'url', 'nummer', 'pfad', 'scrape_datum', 'hochschule_id', 'hochschule_name', 'fakultaet_id', 'fakultaet_name', 'organisation_id', 'organisation_name', 'semester_id', 'semester_name', 'sprache_id', 'sprache_name', 'kursformat_id', 'kursformat_name', 'matchingart_id', 'matchingart_name', 'lehr_und_forschungsbereich_id', 'bio_info', 'ingen_info', 'medien_info', 'medizin_info', 'wirt_info', 'recht_info', 'geo_info', 'bau_info', 'base_info', 'model_1_b', 'model_2', 'model_3', 'model_4']


## Stopwords

In [4]:
from utils import stopwords_config

irrelevant_terms = stopwords_config.irrelevant_terms

sw = list(stopwords.get_stopwords("en"))
sw.extend(list(stopwords.get_stopwords("de")))
sw.extend(irrelevant_terms)
irrelevant_terms

['vl',
 'übung',
 'übungen',
 'seminar',
 'arbeitsgruppenseminar',
 'oberseminar',
 'proseminar',
 'blockveranstaltung',
 'vorlesung',
 'kolloquium',
 'theoriekolloquium',
 'einführung',
 'tutorium',
 'ue',
 'vereinbarung',
 'projekt',
 'praktikum',
 'masterprojekt',
 'wiederholerklausur',
 'fortgeschrittenenpraktikum',
 'hauptseminar',
 'fachpraktikum',
 'ergänzungsvorlesung',
 'forschungspraktikum',
 'begleitseminar',
 'abschlussarbeiten',
 'unterrichtspraktikum',
 'masterseminar',
 'proseminare',
 'praxisseminar',
 'praxissemester',
 'schulpraxis',
 'ringpraktikum',
 'basispraktikum',
 'praxistage',
 'industriepraktikum',
 'vorkurs',
 'projektseminar',
 'juniorprofessur',
 'masterarbeiten',
 'forschungsseminar',
 'modulbeschreibung',
 'veranstaltung',
 'kommentare',
 'raum',
 'uhrzeit',
 'vereinbarung',
 'vorlesung',
 'praktikum',
 'masterprojekt',
 'wiederholerklausur',
 'fortgeschrittenenpraktikum',
 'hauptseminar',
 'fachpraktikum',
 'ergänzungsvorlesung',
 'forschungspraktikum',

# Model

Wir definieren unser Model.

In [5]:
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2", device="cuda" if torch.cuda.is_available() else "cpu")
print("CUDA available:", torch.cuda.is_available())
print("Current device:", model.device)

CUDA available: True
Current device: cuda:0


## Model-Settings, GPU-Check
Konfiguration frei wählbar (einfach im Code unten anpassen).
Hier kann eine erste explorative Untersuchung durchgeführt werden.

Wir checken weiterhin, ob wirklich unsere GPU verwendet wird. Wenn `cuda:0` ist dies der Fall.

In [23]:
# CountVectorizer
vectorizer = CountVectorizer(
  stop_words=sw,  # Entfernt Stopwörter basierend auf der angegebenen Liste (sw)
  token_pattern=r'\b\w+\b',  # Extrahiert nur ganze Wörter, d. h. keine Sonderzeichen oder Zahlen
  ngram_range=(1, 3)  # Erstellt 1-Gramme (einzelne Wörter) bis 3-Gramme (Wortgruppen aus bis zu 3 aufeinanderfolgenden Wörtern)
)

# Embedding Settings  
embedding_model = SentenceTransformer(
    "paraphrase-multilingual-mpnet-base-v2",
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# UMAP Settings
umap_model = UMAP(
    n_neighbors=10,
    n_components=10,
    min_dist=0.0,
    metric="cosine",
    random_state=seed
)
# HDBSCAN Settings
hdbscan_model = HDBSCAN(
    min_cluster_size=100,
    cluster_selection_epsilon=0.2,
    prediction_data=True
)
# Representation Settings
representation_model = MaximalMarginalRelevance(diversity=0.1)

# BERTopic initialisieren
topic_model_1 = BERTopic(
  embedding_model=embedding_model,
  # min_topic_size=100,
  nr_topics="auto", 
  language="multilingual",
  verbose=True,
  umap_model=umap_model,
  vectorizer_model=vectorizer,
  hdbscan_model=hdbscan_model,
  top_n_words = 20,
  representation_model=representation_model
)

## Training

Wir trainieren das Modell.

In [ ]:
topic_model_1.fit(docs)
topic_model_1.get_topic_info()

2026-02-05 14:43:41,506 - BERTopic - Embedding - Transforming documents to embeddings.
Batches:   0%|          | 3/1194 [00:00<01:35, 12.53it/s]

## Outlier Reduzieren, Topics mergen

In [ ]:
topic_model_1.visualize_topics()
#topic_model_1.visualize_hierarchy() # Hilft extrem zu sehen, welche Themen zusammengehören

In [ ]:
# topic_model.merge_topics(docs, [0, 13])

# Outlier reduzieren
# BERTopic auf Test-Daten anwenden
# topics, probs = topic_model_1.transform(docs)
# print(topic_model_1.get_topic_freq())

# # Outlier reduzieren
# topics = topic_model_1.reduce_outliers(docs, topics)

In [10]:
topic_model_1.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,7179,-1_informatik_data_learning_systems,"[informatik, data, learning, systems, software...","[Kolloquium - Informatik , Security and Privac..."
1,0,25371,0_data_informatik_systems_software,"[data, informatik, systems, software, learning...","[Grundlagen der Informatik I , Bachelor-Projek..."
2,1,2005,1_bioinformatik_biometrie_medizinische informa...,"[bioinformatik, biometrie, medizinische inform...","[Querschnittsbereich Epidemiologie, medizinisc..."
3,2,1343,2_security_cryptography_kryptographie_secure,"[security, cryptography, kryptographie, secure...",[Introduction to Modern Cryptography IMPORTANT...
4,3,616,3_operations research_optimization_optimierung...,"[operations research, optimization, optimierun...",[Vorlesung: Grundlagen des Operations Research...
5,4,319,4_geo_geoinformationssysteme_geoinformatik_geo...,"[geo, geoinformationssysteme, geoinformatik, g...",[Statistical Data Analysis Grundlagen der stat...
6,5,278,5_privatissime_privatissime 2_privatissime 2 s...,"[privatissime, privatissime 2, privatissime 2 ...",[Doktorandenseminar (privatissime) 2 Stunden n...
7,6,253,6_algebra diskrete strukturen_algebra diskrete...,"[algebra diskrete strukturen, algebra diskrete...","[Lineare Algebra und Diskrete Strukturen 1 , ..."
8,7,227,7_quantum_computers_computing_quantum computing,"[quantum, computers, computing, quantum comput...","[Grundlagen der digitalen, analogen und Quante..."
9,8,159,8_fuzzy_algorithms_fuzzy methoden_pattern reco...,"[fuzzy, algorithms, fuzzy methoden, pattern re...",[Inf-AlgSeq: Algorithms on Sequences This cour...


## Labeling

Mit Hilfe von gpt4o lassen wir uns die Topcis labeln. Das erleichtert die Interpretation des Modells

In [12]:
from openai import OpenAI as OpenAIClient
from bertopic.representation import OpenAI as OpenAIRep

client = OpenAIClient()  # nutzt OPENAI_API_KEY aus der Umgebung
rep = OpenAIRep(
    client=client, 
    model="gpt-4o-mini", 
    delay_in_seconds=10,      # Erhöhe Delay
    exponential_backoff=True, # Aktiviere das schrittweise längere Warten
    nr_docs=3                 # Weniger Dokumente pro Topic reduzieren die Token-Last
)
topic_model_1.update_topics(docs, representation_model=rep)
topic_model_1.get_topic_info() 

100%|██████████| 20/20 [03:39<00:00, 10.98s/it]


,Topic,Count,Name,Representation,Representative_Docs
0,-1,7179,-1_Data Science Seminars,[Data Science Seminars],"[Kolloquium - Informatik , Security and Privac..."
1,0,25371,0_Scientific research methods,[Scientific research methods],"[Grundlagen der Informatik I , Bachelor-Projek..."
2,1,2005,1_Bioinformatics and Epidemiology,[Bioinformatics and Epidemiology],"[Querschnittsbereich Epidemiologie, medizinisc..."
3,2,1343,2_Introduction to Cryptography,[Introduction to Cryptography],[Introduction to Modern Cryptography IMPORTANT...
4,3,616,3_Operations Research Exercises,[Operations Research Exercises],[Vorlesung: Grundlagen des Operations Research...
5,4,319,4_Software Engineering Overview,[Software Engineering Overview],[Statistical Data Analysis Grundlagen der stat...
6,5,278,5_Doctoral seminar arrangements,[Doctoral seminar arrangements],[Doktorandenseminar (privatissime) 2 Stunden n...
7,6,253,6_Linear Algebra Structures,[Linear Algebra Structures],"[Lineare Algebra und Diskrete Strukturen 1 , ..."
8,7,227,7_Foundations of Computing,[Foundations of Computing],"[Grundlagen der digitalen, analogen und Quante..."
9,8,159,8_Algorithms on Sequences,[Algorithms on Sequences],[Inf-AlgSeq: Algorithms on Sequences This cour...


In [ ]:
# Speichern
topic_model.save("models/explor_model_1", serialization="safetensors", save_embedding_model=True)

# Laden
from bertopic import BERTopic
loaded_model = BERTopic.load("models/explor_model_1")